# Notebook 3: RAD-seq assembly

In this notebook we assemble two RAD-seq libraries prepared by Floragenex using the original RAD method. Data are assembled with *ipyrad* by mapping reads to a reference genome to produce output files for downstream analyses. 

In [3]:
import ipyrad as ip

### PATH to demultiplexed fastq files from each library

In [7]:
FASTQS_1 = "../data_fastqs/demux_lib1_fastqs/*.gz"
FASTQS_2 = "../data_fastqs/demux_lib2_fastqs/*.gz"
FASTQS_3 = "../data_fastqs/demux_digested/*.gz"

### PATH to reference assembly scaffolds fasta file

In [8]:
REF = "../data_ref_genomes/S_irio.fa"

### Load all fastq files 

In [11]:
data1 = ip.Assembly("lib1")
data1.params.sorted_fastq_path = FASTQS_1
data1.run("1", force=True, quiet=True, auto=True)

data2 = ip.Assembly("lib2")
data2.params.sorted_fastq_path = FASTQS_2
data2.run("1", force=True, quiet=True, auto=True)

data3 = ip.Assembly("lib3")
data3.params.sorted_fastq_path = FASTQS_3
data3.run("1", force=True, quiet=True, auto=True)

New Assembly: lib1
New Assembly: lib2
New Assembly: lib3


### Merge samples into one assembly object

In [12]:
data = ip.merge("merged", [data1, data2, data3])

New Assembly: merged



### Set assembly parameters

In [15]:
data.params.project_dir = "../analysis-ipyrad"
data.params.filter_adapters = 3
data.params.assembly_method = "reference"
data.params.reference_sequence = REF
data.params.mindepth_majrule = 2
data.params

0   assembly_name               merged                                       
1   project_dir                 ~/Documents/CachoRAD/analysis-ipyrad         
2   raw_fastq_path              Merged: lib1, lib2, lib3                     
3   barcodes_path               Merged: lib1, lib2, lib3                     
4   sorted_fastq_path           Merged: lib1, lib2, lib3                     
5   assembly_method             reference                                    
6   reference_sequence          ~/Documents/CachoRAD/data_ref_genomes/S_irio.fa
7   datatype                    rad                                          
8   restriction_overhang        ('TGCAG', '')                                
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         33                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            2                             

### Run the assembly steps 2-6

In [16]:
data.ipcluster['cores'] = 38
data.ipcluster['threads'] = 4
data.run("23456", auto=True)

Parallel connection | sacra: 38 cores
[####################] 100% 0:00:01 | concatenating inputs | s2 |
[####################] 100% 0:34:27 | processing reads     | s2 |
[####################] 100% 0:04:01 | indexing reference   | s3 |
[####################] 100% 0:01:47 | dereplicating        | s3 |
[####################] 100% 0:10:17 | mapping reads        | s3 |
[####################] 100% 0:05:40 | building clusters    | s3 |
[####################] 100% 0:00:10 | calc cluster stats   | s3 |
[####################] 100% 0:04:09 | inferring [H, E]     | s4 |
[####################] 100% 0:00:09 | calculating depths   | s5 |
[####################] 100% 0:00:13 | chunking clusters    | s5 |
[####################] 100% 0:29:02 | consens calling      | s5 |
[####################] 100% 0:01:05 | indexing alleles     | s5 |
[####################] 100% 0:00:29 | concatenating bams   | s6 |
[####################] 100% 0:00:08 | fetching regions     | s6 |
[####################] 100% 0:00:15 | 

### Run final assembly step to generate output files

In [25]:
data.stats[data.stats.reads_consens < 1000]

,state,reads_raw,reads_passed_filter,refseq_mapped_reads,refseq_unmapped_reads,clusters_total,clusters_hidepth,hetero_est,error_est,reads_consens
S_campestris_Burgess_1353,6,2108,1495,1081,414,99,77,0.000452,0.002361,77
S_campestris_Rebman_9956,6,14013,5861,4319,1542,345,301,0.000453,0.002291,298
S_cordatus_Stoughton_2201,6,19957,19188,11760,7428,1014,866,0.000937,0.002166,855


In [26]:
keep = data.stats[data.stats.reads_consens > 1000].index.tolist()
min4 = data.branch("Strept_min4", subsamples=keep)

In [27]:
min4.run("7", auto=True)

Parallel connection | sacra: 38 cores
[####################] 100% 0:00:23 | applying filters     | s7 |
[####################] 100% 0:03:09 | building arrays      | s7 |
[####################] 100% 0:02:03 | writing conversions  | s7 |
Parallel connection closed.


In [29]:
min4.stats_dfs.s7_samples

,sample_coverage
reference,40999
A_thaliana_TAIR10,2053
C_amp_amplexicaulis_e068,7878
C_amp_barbarae_REF,3797
C_amp_barbarae_e020,5953
...,...
St_elata_e093,7048
St_pinnata_e090,9879
Sy_irio_NJ_3877,5883
The_crispum_e084,6137
